# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
'''%%bash 
pip install duckdb
pip install jupysql
pip install duckdb-engine'''

Couldn't find program: 'bash'


Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:


In [2]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('./csv/competencia_02_crudo.csv.gz')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [2]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('./csv/competencia_02_b6.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [3]:
%%sql

select * from competencia_02

,numero_de_cliente,foto_mes,cliente_antiguedad,Visa_Master_saldototal,Master_Fvencimiento,Visa_Fvencimiento,tc_fvencimiento_mayor,tc_fvencimiento_menor,Visa_Master_consumo,Visa_Master_finlim,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo:1
0,106324078,201909,107,14202.77,NaN,-1368.0,-1368.0,-1368.0,16376.69,76010.4,...,0.0,12.0,16683.18,-22498.22,0.0,3035.0,5346.81,7.0,0.0,1466.25
1,106324078,201911,109,11851.04,-1826.0,-1307.0,-1307.0,-1826.0,13670.16,354715.2,...,0.0,10.0,13901.26,-16944.33,0.0,3096.0,4631.84,10.0,0.0,1290.30
2,106324078,201912,110,19641.65,-1795.0,-1276.0,-1276.0,-1795.0,50184.56,354715.2,...,0.0,13.0,23039.65,-13901.26,0.0,3127.0,11537.34,15.0,0.0,0.00
3,106324078,202001,111,13578.70,-1764.0,-1245.0,-1245.0,-1764.0,15647.44,354715.2,...,0.0,9.0,15927.81,-23039.65,0.0,3158.0,6659.10,5.0,0.0,1313.76
4,106324078,202002,112,12540.89,-1735.0,-1216.0,-1216.0,-1735.0,14444.52,354715.2,...,0.0,10.0,0.00,-15927.81,0.0,3187.0,5452.13,6.0,0.0,1255.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4562805,116891300,202009,96,21078.83,-790.0,-1795.0,-790.0,-1795.0,26200.47,304041.6,...,0.0,0.0,0.00,-25404.55,0.0,2604.0,9091.72,9.0,0.0,1923.72
4562806,74447632,202003,177,16945.60,-973.0,-243.0,-243.0,-973.0,17666.78,164689.2,...,0.0,13.0,0.00,-8089.41,0.0,2695.0,13706.83,7.0,0.0,3225.75
4562807,95223465,202004,143,16179.01,-91.0,-91.0,-91.0,-91.0,18220.58,354715.2,...,0.0,1.0,0.00,-15941.84,0.0,4307.0,12078.08,9.0,0.0,3014.61
4562808,106572824,201911,109,167237.61,-427.0,-427.0,-427.0,-427.0,48407.17,354715.2,...,0.0,3.0,0.00,-18768.00,0.0,2498.0,38758.03,27.0,0.0,20644.80


Y ya podemos usar **SQL** dentro de una notebook!

In [3]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over part_mes as mes_1
        , lead(mes_0, 2) over part_mes as mes_2
        , lead(mes_0, 3) over part_mes as mes_3
        , lead(mes_0, 4) over part_mes as mes_4
        , lead(mes_0, 5) over part_mes as mes_5
        , lead(mes_0, 6) over part_mes as mes_6
        , CASE 
            WHEN mes_0 = 1 AND mes_1 = 0 AND mes_2=0 AND mes_3=0 AND mes_4=0 AND mes_5=0 AND mes_6=0 THEN 'baja+1'            
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2=0 AND mes_3=0 AND mes_4=0 AND mes_5=0 AND mes_6=0 THEN 'baja+2'
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2=1 AND mes_3=0 AND mes_4=0 AND mes_5=0 AND mes_6=0 THEN 'baja+3'
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2=1 AND mes_3=1 AND mes_4=0 AND mes_5=0 AND mes_6=0 THEN 'baja+4'
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2=1 AND mes_3=1 AND mes_4=1 AND mes_5=0 AND mes_6=0 THEN 'baja+5'
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2=1 AND mes_3=1 AND mes_4=1 AND mes_5=1 AND mes_6=0 THEN 'baja+6'
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2=1 AND mes_3=1 AND mes_4=1 AND mes_5=1 AND mes_6=1 THEN 'continua'
            ELSE 'continua'
          END as clase_ternaria 
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
    WINDOW part_mes AS (partition by t.numero_de_cliente order by foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1

,Success


In [4]:
%%sql
SELECT clase_ternaria, COUNT(*) AS conteo
FROM targets
WHERE foto_mes NOT IN ('202107','202106')
GROUP BY clase_ternaria



,clase_ternaria,conteo
0,continua,4142554
1,baja+6,15825
2,baja+5,15534
3,baja+4,15212
4,baja+3,15043
5,baja+2,14812
6,baja+1,14728


In [136]:
%%sql

SELECT * from targets

,foto_mes,numero_de_cliente,clase_ternaria
0,201901,29185646,continua
1,201902,29185646,continua
2,201903,29185646,continua
3,201904,29185646,continua
4,201905,29185646,continua
...,...,...,...
4562805,202106,186418975,continua
4562806,202107,186418975,continua
4562807,202106,187076825,continua
4562808,202107,187076825,continua


In [21]:
%%sql

select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , mcaja_ahorro
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro

from competencia_02
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,mcaja_ahorro,lag_1_ctrx_quarter,lag_1_mcaja_ahorro
0,29188883,201901,43,0.00,NaN,NaN
1,29188883,201902,45,0.00,43.0,0.00
2,29188883,201903,39,0.00,45.0,0.00
3,29188883,201904,38,9.87,39.0,0.00
4,29188883,201905,39,64.62,38.0,9.87
5,29188883,201906,41,0.00,39.0,64.62
6,29188883,201907,34,0.00,41.0,0.00
7,29188883,201908,30,7.16,34.0,0.00
8,29188883,201909,25,0.00,30.0,7.16
9,29188883,201910,30,0.00,25.0,0.00


In [40]:
%%sql
CREATE MACRO sum_null(a,b) AS COALESCE(a,0)+COALESCE(b,0); --Suma con nulls 

,Success


In [53]:
%%sql
select 
  numero_de_cliente
  , foto_mes
  ,Visa_msaldototal
  ,Master_msaldototal
  , sum_null(Visa_msaldototal,Master_msaldototal) as Visa_Master_saldototal
  , Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
  
  , mtarjeta_master_consumo
  , mtarjeta_visa_consumo
  , sum_null(mtarjeta_master_consumo,mtarjeta_visa_consumo) as Visa_Master_consumo
  , Visa_mfinanciacion_limite 
  , Master_mfinanciacion_limite
  , sum_null(Visa_mfinanciacion_limite,Master_mfinanciacion_limite) as Visa_Master_finlim
  , Visa_msaldopesos 
  , Master_msaldopesos
  , sum_null(Visa_msaldopesos,Master_msaldopesos) as Visa_Master_saldopesos
  , ctarjeta_master
  , ctarjeta_visa
  , sum_null(ctarjeta_master,ctarjeta_visa) as Visa_Master_ctarjetas  
  
FROM competencia_02

LIMIT 10

,numero_de_cliente,foto_mes,Visa_msaldototal,Master_msaldototal,Visa_Master_saldototal,Master_Fvencimiento,Visa_Fvencimiento,tc_fvencimiento_mayor,tc_fvencimiento_menor,mtarjeta_master_consumo,...,Visa_Master_consumo,Visa_mfinanciacion_limite,Master_mfinanciacion_limite,Visa_Master_finlim,Visa_msaldopesos,Master_msaldopesos,Visa_Master_saldopesos,ctarjeta_master,ctarjeta_visa,Visa_Master_ctarjetas
0,29183981,201901,46907.58,45513.70,92421.28,-1429.0,-1429,-1429,-1429,45966.59,...,100743.76,542893.72,542893.72,1085787.44,53401.58,44899.96,98301.54,1,1,2
1,29184630,201901,27741.90,29609.15,57351.05,-1460.0,-1825,-1460,-1825,29609.15,...,62052.37,412395.42,275235.34,687630.76,29277.89,29609.15,58887.04,1,1,2
2,29185587,201901,2862.46,-39890.79,-37028.33,-1368.0,-2007,-1368,-2007,3578.70,...,8748.95,537595.42,538248.64,1075844.06,3357.66,-2423.47,934.19,1,1,2
3,29185646,201901,59375.23,0.00,59375.23,-1245.0,-1368,-1245,-1368,0.00,...,71135.23,538248.64,416459.80,954708.44,69466.20,0.00,69466.20,1,1,2
4,29187499,201901,6762.77,0.00,6762.77,-180.0,-88,-88,-180,0.00,...,7817.07,25336.80,7917.75,33254.55,7932.72,0.00,7932.72,1,1,2
5,29188277,201901,11721.85,NaN,11721.85,NaN,-1337,-1337,-1337,0.00,...,13546.53,63342.00,NaN,63342.00,13749.73,NaN,13749.73,0,1,1
6,29188490,201901,142379.46,62811.74,205191.20,-1245.0,-1946,-1245,-1946,44822.51,...,210384.83,485862.46,485862.46,971724.92,128849.74,45733.52,174583.26,1,1,2
7,29188883,201901,956.35,0.00,956.35,-1276.0,-2007,-1276,-2007,0.00,...,1102.50,416459.80,343670.09,760129.89,1121.79,0.00,1121.79,1,1,2
8,29189950,201901,24312.66,176566.83,200879.49,-1398.0,-1398,-1398,-1398,176566.83,...,205085.58,481120.75,481120.75,962241.50,28518.75,176566.83,205085.58,1,1,2
9,29190599,201901,7386.70,12265.59,19652.29,-1245.0,-1641,-1245,-1641,11358.82,...,19183.60,275235.34,207286.69,482522.03,8664.59,12265.59,20930.18,1,1,2


In [141]:
#Lista de columnas modificadas: (A remover)
para_remover = ("Visa_msaldototal","Master_msaldototal", #Para suma de saldo en tarjetas
              "ctarjeta_master","ctarjeta_visa", #Para sumatotal de tarjetas
              "mtarjeta_master_consumo","mtarjeta_visa_consumo", #Para suma total de consumos
              "Visa_msaldopesos","Master_msaldopesos", #Para suma total de saldos
              "Visa_mfinanciacion_limite","Master_mfinanciacion_limite" #Para suma total de financiació
              )

#Lista de columnas creadas: 
lista_creadas = ("Visa_Master_saldototal", "tc_fvencimiento_mayor", "tc_fvencimiento_menor","Visa_Master_consumo", "Visa_Master_finlim", "Visa_Master_saldopesos", "Visa_Master_ctarjetas") #creadas como sumas


In [74]:
variables_add = ["ctrx_quarter","mcaja_ahorro","mrentabilidad_annual","cpayroll_trx","mpasivos_margen","mprestamos_personales",
                 "mcuentas_saldo","mcuenta_corriente","Visa_mpagominimo","ccomisiones_mantenimiento",
                 "mcomisiones_otras", "cplazo_fijo"
                 ]

In [68]:
campos = ['ctrx_quarter']
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_6 as ctrx_{campo}_slope_6"
print(nuevos_features)



, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_ctrx_quarter_slope_6


In [75]:
campos = variables_add
nuevos_features = ""
for campo in campos:
    nuevos_features += f"\n, {campo} \n, lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}\n, {campo} - lag_1_{campo} as delta_1_{campo}"
    nuevos_features += f"\n, {campo} -lag({campo},2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo}\n, {campo} -lag({campo},3) over (partition by numero_de_cliente order by foto_mes) as lag_3_{campo}"
    nuevos_features += f"\n, {campo} -lag({campo},4) over (partition by numero_de_cliente order by foto_mes) as lag_4_{campo}\n, avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_{campo}"
    nuevos_features += f"\n, avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as avg_4_{campo}"
    nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as {campo}_slope_6"
    nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as {campo}_slope_4"
print(nuevos_features)


, ctrx_quarter 
, lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
, ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
, ctrx_quarter -lag(ctrx_quarter,2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
, ctrx_quarter -lag(ctrx_quarter,3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
, ctrx_quarter -lag(ctrx_quarter,4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ctrx_quarter
, avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctrx_quarter
, avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as avg_4_ctrx_quarter
, regr_slope(ctrx_quarter, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as ctrx_quarter_slope_6
, regr_slope(ctrx_quarter, cliente

In [126]:
primera_ronda = variables_add

primera_todas_col = []
for i in primera_ronda:
    primera_todas_col.append("lag_1_"+i)
    primera_todas_col.append("lag_2_"+i)
    primera_todas_col.append("lag_3_"+i)
    primera_todas_col.append("lag_4_"+i)
    primera_todas_col.append("delta_1_"+i)
    primera_todas_col.append("avg_6_"+i)
    primera_todas_col.append("avg_4_"+i)
    primera_todas_col.append(i+"_slope_6")
    primera_todas_col.append(i+"_slope_4")
print(primera_todas_col)

['lag_1_ctrx_quarter', 'lag_2_ctrx_quarter', 'lag_3_ctrx_quarter', 'lag_4_ctrx_quarter', 'delta_1_ctrx_quarter', 'avg_6_ctrx_quarter', 'avg_4_ctrx_quarter', 'ctrx_quarter_slope_6', 'ctrx_quarter_slope_4', 'lag_1_mcaja_ahorro', 'lag_2_mcaja_ahorro', 'lag_3_mcaja_ahorro', 'lag_4_mcaja_ahorro', 'delta_1_mcaja_ahorro', 'avg_6_mcaja_ahorro', 'avg_4_mcaja_ahorro', 'mcaja_ahorro_slope_6', 'mcaja_ahorro_slope_4', 'lag_1_mrentabilidad_annual', 'lag_2_mrentabilidad_annual', 'lag_3_mrentabilidad_annual', 'lag_4_mrentabilidad_annual', 'delta_1_mrentabilidad_annual', 'avg_6_mrentabilidad_annual', 'avg_4_mrentabilidad_annual', 'mrentabilidad_annual_slope_6', 'mrentabilidad_annual_slope_4', 'lag_1_cpayroll_trx', 'lag_2_cpayroll_trx', 'lag_3_cpayroll_trx', 'lag_4_cpayroll_trx', 'delta_1_cpayroll_trx', 'avg_6_cpayroll_trx', 'avg_4_cpayroll_trx', 'cpayroll_trx_slope_6', 'cpayroll_trx_slope_4', 'lag_1_mpasivos_margen', 'lag_2_mpasivos_margen', 'lag_3_mpasivos_margen', 'lag_4_mpasivos_margen', 'delta_1_mp

In [137]:
%%sql
create or replace table nuevas_col as
select
  numero_de_cliente
  , foto_mes  
  , cliente_antiguedad
  ,Visa_msaldototal --Suma de columnas
  ,Master_msaldototal
  , sum_null(Visa_msaldototal,Master_msaldototal) as Visa_Master_saldototal
  , Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
  
  , mtarjeta_master_consumo
  , mtarjeta_visa_consumo
  , sum_null(mtarjeta_master_consumo,mtarjeta_visa_consumo) as Visa_Master_consumo
  , Visa_mfinanciacion_limite 
  , Master_mfinanciacion_limite
  , sum_null(Visa_mfinanciacion_limite,Master_mfinanciacion_limite) as Visa_Master_finlim
  , Visa_msaldopesos 
  , Master_msaldopesos
  , sum_null(Visa_msaldopesos,Master_msaldopesos) as Visa_Master_saldopesos
  , ctarjeta_master
  , ctarjeta_visa
  , sum_null(ctarjeta_master,ctarjeta_visa) as Visa_Master_ctarjetas --creación de parametros
    
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as ctrx_quarter_slope_6
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as ctrx_quarter_slope_4

  , mcaja_ahorro
  , lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
  , mcaja_ahorro - lag_1_mcaja_ahorro as delta_1_mcaja_ahorro
  , mcaja_ahorro - lag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro
  , mcaja_ahorro - lag(mcaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro
  , mcaja_ahorro - lag(mcaja_ahorro, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcaja_ahorro
  , avg(mcaja_ahorro) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_mcaja_ahorro
  , avg(mcaja_ahorro) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_mcaja_ahorro
  ,regr_slope(mcaja_ahorro, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as mcaja_ahorro_slope_6
  ,regr_slope(mcaja_ahorro, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as mcaja_ahorro_slope_4
  
  , mrentabilidad_annual
  , lag(mrentabilidad_annual, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual
  , mrentabilidad_annual - lag_1_mrentabilidad_annual as delta_1_mrentabilidad_annual
  , mrentabilidad_annual - lag(mrentabilidad_annual, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mrentabilidad_annual
  , mrentabilidad_annual - lag(mrentabilidad_annual, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad_annual
  , mrentabilidad_annual - lag(mrentabilidad_annual, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mrentabilidad_annual
  , avg(mrentabilidad_annual) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_mrentabilidad_annual
  , avg(mrentabilidad_annual) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_mrentabilidad_annual  
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as mrentabilidad_annual_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as mrentabilidad_annual_slope_4
  
  , cpayroll_trx
  , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
  , cpayroll_trx - lag_1_mcaja_ahorro as delta_1_cpayroll_trx
  , cpayroll_trx - lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
  , cpayroll_trx - lag(cpayroll_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll_trx
  , cpayroll_trx - lag(cpayroll_trx, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_cpayroll_trx
  , avg(cpayroll_trx) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_cpayroll_trx
  , avg(cpayroll_trx) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_cpayroll_trx
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as cpayroll_trx_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as cpayroll_trx_slope_4  
  
  , mpasivos_margen
  , lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
  , mpasivos_margen - lag_1_mcaja_ahorro as delta_1_mpasivos_margen
  , mpasivos_margen - lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpasivos_margen
  , mpasivos_margen - lag(mpasivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpasivos_margen
  , mpasivos_margen - lag(mpasivos_margen, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mpasivos_margen
  , avg(mpasivos_margen) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_mpasivos_margen
  , avg(mpasivos_margen) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_mpasivos_margen
,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as mpasivos_margen_slope_6
,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as mpasivos_margen_slope_4  

, mprestamos_personales
  , lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
  , mprestamos_personales - lag_1_mcaja_ahorro as delta_1_mprestamos_personales
  , mprestamos_personales - lag(mprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_personales
  , mprestamos_personales - lag(mprestamos_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_personales
  , mprestamos_personales - lag(mprestamos_personales, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mprestamos_personales
  , avg(mprestamos_personales) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_mprestamos_personales
  , avg(mprestamos_personales) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_mprestamos_personales
,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as mprestamos_personales_slope_6
,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as mprestamos_personales_slope_4 

, cpayroll_trx
  , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
  , cpayroll_trx - lag_1_mcaja_ahorro as delta_1_cpayroll_trx
  , cpayroll_trx - lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
  , cpayroll_trx - lag(cpayroll_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll_trx
  , cpayroll_trx - lag(cpayroll_trx, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_cpayroll_trx
  , avg(cpayroll_trx) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_cpayroll_trx
  , avg(cpayroll_trx) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_cpayroll_trx
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as cpayroll_trx_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as cpayroll_trx_slope_4  

, mcuentas_saldo
  , lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
  , mcuentas_saldo - lag_1_mcaja_ahorro as delta_1_mcuentas_saldo
  , mcuentas_saldo - lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuentas_saldo
  , mcuentas_saldo - lag(mcuentas_saldo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuentas_saldo
  , mcuentas_saldo - lag(mcuentas_saldo, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcuentas_saldo
  , avg(mcuentas_saldo) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_mcuentas_saldo
  , avg(mcuentas_saldo) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_mcuentas_saldo
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as mcuentas_saldo_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as mcuentas_saldo_slope_4   

,mcuenta_corriente
  , lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
  , mcuenta_corriente - lag_1_mcaja_ahorro as delta_1_mcuenta_corriente
  , mcuenta_corriente - lag(mcuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente
  , mcuenta_corriente - lag(mcuenta_corriente, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_corriente
  , mcuenta_corriente - lag(mcuenta_corriente, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcuenta_corriente
  , avg(mcuenta_corriente) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_mcuenta_corriente
  , avg(mcuenta_corriente) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_mcuenta_corriente
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as mcuenta_corriente_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as mcuenta_corriente_slope_4 

,  Visa_mpagominimo
  , lag(Visa_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo
  , Visa_mpagominimo - lag_1_mcaja_ahorro as delta_1_Visa_mpagominimo
  , Visa_mpagominimo - lag(Visa_mpagominimo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagominimo
  , Visa_mpagominimo - lag(Visa_mpagominimo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagominimo
  , Visa_mpagominimo - lag(Visa_mpagominimo, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_mpagominimo
  , avg(Visa_mpagominimo) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_Visa_mpagominimo
  , avg(Visa_mpagominimo) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_Visa_mpagominimo
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as Visa_mpagominimo_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as Visa_mpagominimo_slope_4 

,  ccomisiones_mantenimiento
  , lag(ccomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_mantenimiento
  , ccomisiones_mantenimiento - lag_1_mcaja_ahorro as delta_1_ccomisiones_mantenimiento
  , ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccomisiones_mantenimiento
  , ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_mantenimiento
  , ccomisiones_mantenimiento - lag(ccomisiones_mantenimiento, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ccomisiones_mantenimiento
  , avg(ccomisiones_mantenimiento) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_ccomisiones_mantenimiento
  , avg(ccomisiones_mantenimiento) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_ccomisiones_mantenimiento
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as ccomisiones_mantenimiento_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as ccomisiones_mantenimiento_slope_4 

,  mcomisiones_otras
  , lag(mcomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras
  , mcomisiones_otras - lag_1_mcaja_ahorro as delta_1_mcomisiones_otras
  , mcomisiones_otras - lag(mcomisiones_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_otras
  , mcomisiones_otras - lag(mcomisiones_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_otras
  , mcomisiones_otras - lag(mcomisiones_otras, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcomisiones_otras
  , avg(mcomisiones_otras) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_mcomisiones_otras
  , avg(mcomisiones_otras) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_mcomisiones_otras
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as mcomisiones_otras_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as mcomisiones_otras_slope_4 

, cplazo_fijo
  , lag(cplazo_fijo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cplazo_fijo
  , cplazo_fijo - lag_1_mcaja_ahorro as delta_1_cplazo_fijo
  , cplazo_fijo - lag(cplazo_fijo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cplazo_fijo
  , cplazo_fijo - lag(cplazo_fijo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cplazo_fijo
  , cplazo_fijo - lag(cplazo_fijo, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_cplazo_fijo
  , avg(cplazo_fijo) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_cplazo_fijo
  , avg(cplazo_fijo) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 4 preceding and current row) as avg_4_cplazo_fijo
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as cplazo_fijo_slope_6
  ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as cplazo_fijo_slope_4 



from competencia_02

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [100]:
#Segunda ronda con las columnas creadas antes, esto seguramente es sub optimo, pero es lo que hay 
segunda_ronda = ["Visa_Master_finlim","Visa_Master_saldototal","Visa_Master_saldopesos"]
campos = segunda_ronda
nuevos_features = ""
for campo in campos:
    nuevos_features += f"\n, {campo} \n, lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}\n, {campo} - lag_1_{campo} as delta_1_{campo}"
    nuevos_features += f"\n, {campo} -lag({campo},2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo}\n, {campo} -lag({campo},3) over (partition by numero_de_cliente order by foto_mes) as lag_3_{campo}"
    nuevos_features += f"\n, {campo} -lag({campo},4) over (partition by numero_de_cliente order by foto_mes) as lag_4_{campo}\n, avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_{campo}"
    nuevos_features += f"\n, avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as avg_4_{campo}"
    nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as {campo}_slope_6"
    nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as {campo}_slope_4"
print(nuevos_features)

segunda_todas_col = []
for i in segunda_ronda:
    segunda_todas_col.append("lag_1_"+i)
    segunda_todas_col.append("lag_2_"+i)
    segunda_todas_col.append("lag_3_"+i)
    segunda_todas_col.append("lag_4_"+i)
    segunda_todas_col.append("delta_1_"+i)
    segunda_todas_col.append("avg_6_"+i)
    segunda_todas_col.append("avg_4_"+i)
    segunda_todas_col.append(i+"_slope_6")
    segunda_todas_col.append(i+"_slope_4")
print(segunda_todas_col)


, Visa_Master_finlim 
, lag(Visa_Master_finlim, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Master_finlim
, Visa_Master_finlim - lag_1_Visa_Master_finlim as delta_1_Visa_Master_finlim
, Visa_Master_finlim -lag(Visa_Master_finlim,2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Master_finlim
, Visa_Master_finlim -lag(Visa_Master_finlim,3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Master_finlim
, Visa_Master_finlim -lag(Visa_Master_finlim,4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_Master_finlim
, avg(Visa_Master_finlim) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_Master_finlim
, avg(Visa_Master_finlim) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as avg_4_Visa_Master_finlim
, regr_slope(Visa_Master_finlim, cliente_antiguedad) over (partition by numero_de_client

In [138]:
%%sql
create or replace table nuevas_col2 as
select
    numero_de_cliente
    , foto_mes  

, Visa_Master_finlim 
, lag(Visa_Master_finlim, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Master_finlim
, Visa_Master_finlim - lag_1_Visa_Master_finlim as delta_1_Visa_Master_finlim
, Visa_Master_finlim -lag(Visa_Master_finlim,2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Master_finlim
, Visa_Master_finlim -lag(Visa_Master_finlim,3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Master_finlim
, Visa_Master_finlim -lag(Visa_Master_finlim,4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_Master_finlim
, avg(Visa_Master_finlim) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_Master_finlim
, avg(Visa_Master_finlim) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as avg_4_Visa_Master_finlim
, regr_slope(Visa_Master_finlim, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as Visa_Master_finlim_slope_6
, regr_slope(Visa_Master_finlim, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as Visa_Master_finlim_slope_4
, Visa_Master_saldototal 
, lag(Visa_Master_saldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Master_saldototal
, Visa_Master_saldototal - lag_1_Visa_Master_saldototal as delta_1_Visa_Master_saldototal
, Visa_Master_saldototal -lag(Visa_Master_saldototal,2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Master_saldototal
, Visa_Master_saldototal -lag(Visa_Master_saldototal,3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Master_saldototal
, Visa_Master_saldototal -lag(Visa_Master_saldototal,4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_Master_saldototal
, avg(Visa_Master_saldototal) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_Master_saldototal
, avg(Visa_Master_saldototal) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as avg_4_Visa_Master_saldototal
, regr_slope(Visa_Master_saldototal, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as Visa_Master_saldototal_slope_6
, regr_slope(Visa_Master_saldototal, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as Visa_Master_saldototal_slope_4
, Visa_Master_saldopesos 
, lag(Visa_Master_saldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Master_saldopesos
, Visa_Master_saldopesos - lag_1_Visa_Master_saldopesos as delta_1_Visa_Master_saldopesos
, Visa_Master_saldopesos -lag(Visa_Master_saldopesos,2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Master_saldopesos
, Visa_Master_saldopesos -lag(Visa_Master_saldopesos,3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Master_saldopesos
, Visa_Master_saldopesos -lag(Visa_Master_saldopesos,4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_Master_saldopesos
, avg(Visa_Master_saldopesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_Master_saldopesos
, avg(Visa_Master_saldopesos) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as avg_4_Visa_Master_saldopesos
, regr_slope(Visa_Master_saldopesos, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as Visa_Master_saldopesos_slope_6
, regr_slope(Visa_Master_saldopesos, cliente_antiguedad) over (partition by numero_de_cliente order by foto_mes rows between 4 preceding and current row) as Visa_Master_saldopesos_slope_4

from nuevas_col


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [140]:
%%sql
create or replace table para_export as
Select *
FROM nuevas_col
INNER JOIN nuevas_col2
ON nuevas_col.numero_de_cliente = nuevas_col2.numero_de_cliente AND nuevas_col.foto_mes = nuevas_col2.foto_mes
INNER JOIN targets
ON nuevas_col.numero_de_cliente = targets.numero_de_cliente AND nuevas_col.foto_mes = targets.foto_mes
INNER JOIN competencia_02
ON nuevas_col.numero_de_cliente = competencia_02.numero_de_cliente AND nuevas_col.foto_mes = competencia_02.foto_mes



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
#No lo use
columnas_new = segunda_todas_col

sql_query = ""
for i in columnas_new:
    sql_query += f"\n alter table nuevas_col add column {i} double precision;"
print(sql_query)

In [157]:
columnas_new = para_remover

sql_drop_table = ""
for i in columnas_new:
    sql_drop_table += f"ALTER TABLE para_export DROP COLUMN {i};\n "
print(sql_drop_table)


ALTER TABLE para_export DROP COLUMN Visa_msaldototal;
 ALTER TABLE para_export DROP COLUMN Master_msaldototal;
 ALTER TABLE para_export DROP COLUMN ctarjeta_master;
 ALTER TABLE para_export DROP COLUMN ctarjeta_visa;
 ALTER TABLE para_export DROP COLUMN mtarjeta_master_consumo;
 ALTER TABLE para_export DROP COLUMN mtarjeta_visa_consumo;
 ALTER TABLE para_export DROP COLUMN Visa_msaldopesos;
 ALTER TABLE para_export DROP COLUMN Master_msaldopesos;
 ALTER TABLE para_export DROP COLUMN Visa_mfinanciacion_limite;
 ALTER TABLE para_export DROP COLUMN Master_mfinanciacion_limite;
 


In [159]:
%%sql
 ALTER TABLE para_export DROP COLUMN Visa_msaldototal;
 ALTER TABLE para_export DROP COLUMN Master_msaldototal;
 ALTER TABLE para_export DROP COLUMN ctarjeta_master;
 ALTER TABLE para_export DROP COLUMN ctarjeta_visa;
 ALTER TABLE para_export DROP COLUMN mtarjeta_master_consumo;
 ALTER TABLE para_export DROP COLUMN mtarjeta_visa_consumo;
 ALTER TABLE para_export DROP COLUMN Visa_msaldopesos;
 ALTER TABLE para_export DROP COLUMN Master_msaldopesos;
 ALTER TABLE para_export DROP COLUMN Visa_mfinanciacion_limite;
 ALTER TABLE para_export DROP COLUMN Master_mfinanciacion_limite;

,Success


In [ ]:
#Lista de columnas modificadas: (A remover)
para_remover = ("Visa_msaldototal","Master_msaldototal", #Para suma de saldo en tarjetas
              "ctarjeta_master","ctarjeta_visa", #Para sumatotal de tarjetas
              "mtarjeta_master_consumo","mtarjeta_visa_consumo", #Para suma total de consumos
              "Visa_msaldopesos","Master_msaldopesos", #Para suma total de saldos
              "Visa_mfinanciacion_limite","Master_mfinanciacion_limite" #Para suma total de financiació
              )

#Lista de columnas creadas: 
lista_creadas = ("Visa_Master_saldototal", "tc_fvencimiento_mayor", "tc_fvencimiento_menor","Visa_Master_consumo", "Visa_Master_finlim", "Visa_Master_saldopesos", "Visa_Master_ctarjetas") #creadas como sumas

In [164]:
#for para evitar el drifting 

columnas_new = ["Master_mlimitecompra",	"Master_mpagado",	"Master_mpagospesos",	"Master_mconsumototal",	"Master_mpagominimo",	"Visa_mconsumospesos",	
                "Visa_mlimitecompra",	"Visa_mpagado",	"Visa_mpagospesos",	"Visa_mconsumototal",	"Visa_mpagominimo",	"mcomisiones",
                "mactivos_margen",	"mpasivos_margen",	"mcuenta_corriente",	"mprestamos_personales",	"mprestamos_prendarios",	"mprestamos_hipotecarios",	"mplazo_fijo_pesos",	"minversion1_pesos",
                "minversion2",	"mpayroll",	"mpayroll2",	"mcuenta_debitos_automaticos",	"mttarjeta_visa_debitos_automaticos",	"mttarjeta_master_debitos_automaticos",	"mpagodeservicios",	"mpagomiscuentas",
                "mcajeros_propios_descuentos",	"mtarjeta_visa_descuentos",	"mtarjeta_master_descuentos",	"mcomisiones_mantenimiento",	"mcomisiones_otras",	"mtransferencias_recibidas",	"mtransferencias_emitidas",
                "mextraccion_autoservicio",	"mcheques_depositados",	"mcheques_emitidos",	"mcheques_depositados_rechazados",	"mcheques_emitidos_rechazados","Visa_Master_saldototal","Visa_Master_consumo","Visa_Master_finlim",
                "Visa_Master_saldopesos"]

sql_drifting = ""
for i in columnas_new:
    sql_drifting += f",{i}\n, row_number() over (partition by foto_mes order by {i}) as {i}_drift\n, percent_rank() over (partition by foto_mes order by {i}) as {i}_drift_per\n"
print(sql_drifting)


,Master_mlimitecompra
, row_number() over (partition by foto_mes order by Master_mlimitecompra) as Master_mlimitecompra_drift
, percent_rank() over (partition by foto_mes order by Master_mlimitecompra) as Master_mlimitecompra_drift_per
,Master_mpagado
, row_number() over (partition by foto_mes order by Master_mpagado) as Master_mpagado_drift
, percent_rank() over (partition by foto_mes order by Master_mpagado) as Master_mpagado_drift_per
,Master_mpagospesos
, row_number() over (partition by foto_mes order by Master_mpagospesos) as Master_mpagospesos_drift
, percent_rank() over (partition by foto_mes order by Master_mpagospesos) as Master_mpagospesos_drift_per
,Master_mconsumototal
, row_number() over (partition by foto_mes order by Master_mconsumototal) as Master_mconsumototal_drift
, percent_rank() over (partition by foto_mes order by Master_mconsumototal) as Master_mconsumototal_drift_per
,Master_mpagominimo
, row_number() over (partition by foto_mes order by Master_mpagominimo) as M

In [165]:
%%sql
create or replace table drift as
select
    foto_mes
,Master_mlimitecompra
, row_number() over (partition by foto_mes order by Master_mlimitecompra) as Master_mlimitecompra_drift
, percent_rank() over (partition by foto_mes order by Master_mlimitecompra) as Master_mlimitecompra_drift_per
,Master_mpagado
, row_number() over (partition by foto_mes order by Master_mpagado) as Master_mpagado_drift
, percent_rank() over (partition by foto_mes order by Master_mpagado) as Master_mpagado_drift_per
,Master_mpagospesos
, row_number() over (partition by foto_mes order by Master_mpagospesos) as Master_mpagospesos_drift
, percent_rank() over (partition by foto_mes order by Master_mpagospesos) as Master_mpagospesos_drift_per
,Master_mconsumototal
, row_number() over (partition by foto_mes order by Master_mconsumototal) as Master_mconsumototal_drift
, percent_rank() over (partition by foto_mes order by Master_mconsumototal) as Master_mconsumototal_drift_per
,Master_mpagominimo
, row_number() over (partition by foto_mes order by Master_mpagominimo) as Master_mpagominimo_drift
, percent_rank() over (partition by foto_mes order by Master_mpagominimo) as Master_mpagominimo_drift_per
,Visa_mconsumospesos
, row_number() over (partition by foto_mes order by Visa_mconsumospesos) as Visa_mconsumospesos_drift
, percent_rank() over (partition by foto_mes order by Visa_mconsumospesos) as Visa_mconsumospesos_drift_per
,Visa_mlimitecompra
, row_number() over (partition by foto_mes order by Visa_mlimitecompra) as Visa_mlimitecompra_drift
, percent_rank() over (partition by foto_mes order by Visa_mlimitecompra) as Visa_mlimitecompra_drift_per
,Visa_mpagado
, row_number() over (partition by foto_mes order by Visa_mpagado) as Visa_mpagado_drift
, percent_rank() over (partition by foto_mes order by Visa_mpagado) as Visa_mpagado_drift_per
,Visa_mpagospesos
, row_number() over (partition by foto_mes order by Visa_mpagospesos) as Visa_mpagospesos_drift
, percent_rank() over (partition by foto_mes order by Visa_mpagospesos) as Visa_mpagospesos_drift_per
,Visa_mconsumototal
, row_number() over (partition by foto_mes order by Visa_mconsumototal) as Visa_mconsumototal_drift
, percent_rank() over (partition by foto_mes order by Visa_mconsumototal) as Visa_mconsumototal_drift_per
,Visa_mpagominimo
, row_number() over (partition by foto_mes order by Visa_mpagominimo) as Visa_mpagominimo_drift
, percent_rank() over (partition by foto_mes order by Visa_mpagominimo) as Visa_mpagominimo_drift_per
,mcomisiones
, row_number() over (partition by foto_mes order by mcomisiones) as mcomisiones_drift
, percent_rank() over (partition by foto_mes order by mcomisiones) as mcomisiones_drift_per
,mactivos_margen
, row_number() over (partition by foto_mes order by mactivos_margen) as mactivos_margen_drift
, percent_rank() over (partition by foto_mes order by mactivos_margen) as mactivos_margen_drift_per
,mpasivos_margen
, row_number() over (partition by foto_mes order by mpasivos_margen) as mpasivos_margen_drift
, percent_rank() over (partition by foto_mes order by mpasivos_margen) as mpasivos_margen_drift_per
,mcuenta_corriente
, row_number() over (partition by foto_mes order by mcuenta_corriente) as mcuenta_corriente_drift
, percent_rank() over (partition by foto_mes order by mcuenta_corriente) as mcuenta_corriente_drift_per
,mprestamos_personales
, row_number() over (partition by foto_mes order by mprestamos_personales) as mprestamos_personales_drift
, percent_rank() over (partition by foto_mes order by mprestamos_personales) as mprestamos_personales_drift_per
,mprestamos_prendarios
, row_number() over (partition by foto_mes order by mprestamos_prendarios) as mprestamos_prendarios_drift
, percent_rank() over (partition by foto_mes order by mprestamos_prendarios) as mprestamos_prendarios_drift_per
,mprestamos_hipotecarios
, row_number() over (partition by foto_mes order by mprestamos_hipotecarios) as mprestamos_hipotecarios_drift
, percent_rank() over (partition by foto_mes order by mprestamos_hipotecarios) as mprestamos_hipotecarios_drift_per
,mplazo_fijo_pesos
, row_number() over (partition by foto_mes order by mplazo_fijo_pesos) as mplazo_fijo_pesos_drift
, percent_rank() over (partition by foto_mes order by mplazo_fijo_pesos) as mplazo_fijo_pesos_drift_per
,minversion1_pesos
, row_number() over (partition by foto_mes order by minversion1_pesos) as minversion1_pesos_drift
, percent_rank() over (partition by foto_mes order by minversion1_pesos) as minversion1_pesos_drift_per
,minversion2
, row_number() over (partition by foto_mes order by minversion2) as minversion2_drift
, percent_rank() over (partition by foto_mes order by minversion2) as minversion2_drift_per
,mpayroll
, row_number() over (partition by foto_mes order by mpayroll) as mpayroll_drift
, percent_rank() over (partition by foto_mes order by mpayroll) as mpayroll_drift_per
,mpayroll2
, row_number() over (partition by foto_mes order by mpayroll2) as mpayroll2_drift
, percent_rank() over (partition by foto_mes order by mpayroll2) as mpayroll2_drift_per
,mcuenta_debitos_automaticos
, row_number() over (partition by foto_mes order by mcuenta_debitos_automaticos) as mcuenta_debitos_automaticos_drift
, percent_rank() over (partition by foto_mes order by mcuenta_debitos_automaticos) as mcuenta_debitos_automaticos_drift_per
,mttarjeta_visa_debitos_automaticos
, row_number() over (partition by foto_mes order by mttarjeta_visa_debitos_automaticos) as mttarjeta_visa_debitos_automaticos_drift
, percent_rank() over (partition by foto_mes order by mttarjeta_visa_debitos_automaticos) as mttarjeta_visa_debitos_automaticos_drift_per
,mttarjeta_master_debitos_automaticos
, row_number() over (partition by foto_mes order by mttarjeta_master_debitos_automaticos) as mttarjeta_master_debitos_automaticos_drift
, percent_rank() over (partition by foto_mes order by mttarjeta_master_debitos_automaticos) as mttarjeta_master_debitos_automaticos_drift_per
,mpagodeservicios
, row_number() over (partition by foto_mes order by mpagodeservicios) as mpagodeservicios_drift
, percent_rank() over (partition by foto_mes order by mpagodeservicios) as mpagodeservicios_drift_per
,mpagomiscuentas
, row_number() over (partition by foto_mes order by mpagomiscuentas) as mpagomiscuentas_drift
, percent_rank() over (partition by foto_mes order by mpagomiscuentas) as mpagomiscuentas_drift_per
,mcajeros_propios_descuentos
, row_number() over (partition by foto_mes order by mcajeros_propios_descuentos) as mcajeros_propios_descuentos_drift
, percent_rank() over (partition by foto_mes order by mcajeros_propios_descuentos) as mcajeros_propios_descuentos_drift_per
,mtarjeta_visa_descuentos
, row_number() over (partition by foto_mes order by mtarjeta_visa_descuentos) as mtarjeta_visa_descuentos_drift
, percent_rank() over (partition by foto_mes order by mtarjeta_visa_descuentos) as mtarjeta_visa_descuentos_drift_per
,mtarjeta_master_descuentos
, row_number() over (partition by foto_mes order by mtarjeta_master_descuentos) as mtarjeta_master_descuentos_drift
, percent_rank() over (partition by foto_mes order by mtarjeta_master_descuentos) as mtarjeta_master_descuentos_drift_per
,mcomisiones_mantenimiento
, row_number() over (partition by foto_mes order by mcomisiones_mantenimiento) as mcomisiones_mantenimiento_drift
, percent_rank() over (partition by foto_mes order by mcomisiones_mantenimiento) as mcomisiones_mantenimiento_drift_per
,mcomisiones_otras
, row_number() over (partition by foto_mes order by mcomisiones_otras) as mcomisiones_otras_drift
, percent_rank() over (partition by foto_mes order by mcomisiones_otras) as mcomisiones_otras_drift_per
,mtransferencias_recibidas
, row_number() over (partition by foto_mes order by mtransferencias_recibidas) as mtransferencias_recibidas_drift
, percent_rank() over (partition by foto_mes order by mtransferencias_recibidas) as mtransferencias_recibidas_drift_per
,mtransferencias_emitidas
, row_number() over (partition by foto_mes order by mtransferencias_emitidas) as mtransferencias_emitidas_drift
, percent_rank() over (partition by foto_mes order by mtransferencias_emitidas) as mtransferencias_emitidas_drift_per
,mextraccion_autoservicio
, row_number() over (partition by foto_mes order by mextraccion_autoservicio) as mextraccion_autoservicio_drift
, percent_rank() over (partition by foto_mes order by mextraccion_autoservicio) as mextraccion_autoservicio_drift_per
,mcheques_depositados
, row_number() over (partition by foto_mes order by mcheques_depositados) as mcheques_depositados_drift
, percent_rank() over (partition by foto_mes order by mcheques_depositados) as mcheques_depositados_drift_per
,mcheques_emitidos
, row_number() over (partition by foto_mes order by mcheques_emitidos) as mcheques_emitidos_drift
, percent_rank() over (partition by foto_mes order by mcheques_emitidos) as mcheques_emitidos_drift_per
,mcheques_depositados_rechazados
, row_number() over (partition by foto_mes order by mcheques_depositados_rechazados) as mcheques_depositados_rechazados_drift
, percent_rank() over (partition by foto_mes order by mcheques_depositados_rechazados) as mcheques_depositados_rechazados_drift_per
,mcheques_emitidos_rechazados
, row_number() over (partition by foto_mes order by mcheques_emitidos_rechazados) as mcheques_emitidos_rechazados_drift
, percent_rank() over (partition by foto_mes order by mcheques_emitidos_rechazados) as mcheques_emitidos_rechazados_drift_per
,Visa_Master_saldototal
, row_number() over (partition by foto_mes order by Visa_Master_saldototal) as Visa_Master_saldototal_drift
, percent_rank() over (partition by foto_mes order by Visa_Master_saldototal) as Visa_Master_saldototal_drift_per
,Visa_Master_consumo
, row_number() over (partition by foto_mes order by Visa_Master_consumo) as Visa_Master_consumo_drift
, percent_rank() over (partition by foto_mes order by Visa_Master_consumo) as Visa_Master_consumo_drift_per
,Visa_Master_finlim
, row_number() over (partition by foto_mes order by Visa_Master_finlim) as Visa_Master_finlim_drift
, percent_rank() over (partition by foto_mes order by Visa_Master_finlim) as Visa_Master_finlim_drift_per
,Visa_Master_saldopesos
, row_number() over (partition by foto_mes order by Visa_Master_saldopesos) as Visa_Master_saldopesos_drift
, percent_rank() over (partition by foto_mes order by Visa_Master_saldopesos) as Visa_Master_saldopesos_drift_per
from para_export



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

RuntimeError: Query interrupted

In [160]:
%%sql
copy para_export to './csv/competencia_02_b6.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [120]:
#Update de la tabla 


['lag_1_Visa_Master_finlim',
 'lag_2_Visa_Master_finlim',
 'lag_3_Visa_Master_finlim',
 'lag_4_Visa_Master_finlim',
 'delta_1_Visa_Master_finlim',
 'avg_6_Visa_Master_finlim',
 'avg_4_Visa_Master_finlim',
 'Visa_Master_finlim_slope_6',
 'Visa_Master_finlim_slope_4',
 'lag_1_Visa_Master_saldototal',
 'lag_2_Visa_Master_saldototal',
 'lag_3_Visa_Master_saldototal',
 'lag_4_Visa_Master_saldototal',
 'delta_1_Visa_Master_saldototal',
 'avg_6_Visa_Master_saldototal',
 'avg_4_Visa_Master_saldototal',
 'Visa_Master_saldototal_slope_6',
 'Visa_Master_saldototal_slope_4',
 'lag_1_Visa_Master_saldopesos',
 'lag_2_Visa_Master_saldopesos',
 'lag_3_Visa_Master_saldopesos',
 'lag_4_Visa_Master_saldopesos',
 'delta_1_Visa_Master_saldopesos',
 'avg_6_Visa_Master_saldopesos',
 'avg_4_Visa_Master_saldopesos',
 'Visa_Master_saldopesos_slope_6',
 'Visa_Master_saldopesos_slope_4']

In [123]:
#Update de las columnas en nuevas_col
columnas_new = segunda_todas_col

sql_set = "set "
for i in columnas_new:
    sql_set += f"{i} =nuevas_col2.{i}\n,"
print(sql_set)


set lag_1_Visa_Master_finlim =nuevas_col2.lag_1_Visa_Master_finlim
,lag_2_Visa_Master_finlim =nuevas_col2.lag_2_Visa_Master_finlim
,lag_3_Visa_Master_finlim =nuevas_col2.lag_3_Visa_Master_finlim
,lag_4_Visa_Master_finlim =nuevas_col2.lag_4_Visa_Master_finlim
,delta_1_Visa_Master_finlim =nuevas_col2.delta_1_Visa_Master_finlim
,avg_6_Visa_Master_finlim =nuevas_col2.avg_6_Visa_Master_finlim
,avg_4_Visa_Master_finlim =nuevas_col2.avg_4_Visa_Master_finlim
,Visa_Master_finlim_slope_6 =nuevas_col2.Visa_Master_finlim_slope_6
,Visa_Master_finlim_slope_4 =nuevas_col2.Visa_Master_finlim_slope_4
,lag_1_Visa_Master_saldototal =nuevas_col2.lag_1_Visa_Master_saldototal
,lag_2_Visa_Master_saldototal =nuevas_col2.lag_2_Visa_Master_saldototal
,lag_3_Visa_Master_saldototal =nuevas_col2.lag_3_Visa_Master_saldototal
,lag_4_Visa_Master_saldototal =nuevas_col2.lag_4_Visa_Master_saldototal
,delta_1_Visa_Master_saldototal =nuevas_col2.delta_1_Visa_Master_saldototal
,avg_6_Visa_Master_saldototal =nuevas_col2.av

In [124]:
%%sql
update nuevas_col
set lag_1_Visa_Master_finlim =nuevas_col2.lag_1_Visa_Master_finlim
,lag_2_Visa_Master_finlim =nuevas_col2.lag_2_Visa_Master_finlim
,lag_3_Visa_Master_finlim =nuevas_col2.lag_3_Visa_Master_finlim
,lag_4_Visa_Master_finlim =nuevas_col2.lag_4_Visa_Master_finlim
,delta_1_Visa_Master_finlim =nuevas_col2.delta_1_Visa_Master_finlim
,avg_6_Visa_Master_finlim =nuevas_col2.avg_6_Visa_Master_finlim
,avg_4_Visa_Master_finlim =nuevas_col2.avg_4_Visa_Master_finlim
,Visa_Master_finlim_slope_6 =nuevas_col2.Visa_Master_finlim_slope_6
,Visa_Master_finlim_slope_4 =nuevas_col2.Visa_Master_finlim_slope_4
,lag_1_Visa_Master_saldototal =nuevas_col2.lag_1_Visa_Master_saldototal
,lag_2_Visa_Master_saldototal =nuevas_col2.lag_2_Visa_Master_saldototal
,lag_3_Visa_Master_saldototal =nuevas_col2.lag_3_Visa_Master_saldototal
,lag_4_Visa_Master_saldototal =nuevas_col2.lag_4_Visa_Master_saldototal
,delta_1_Visa_Master_saldototal =nuevas_col2.delta_1_Visa_Master_saldototal
,avg_6_Visa_Master_saldototal =nuevas_col2.avg_6_Visa_Master_saldototal
,avg_4_Visa_Master_saldototal =nuevas_col2.avg_4_Visa_Master_saldototal
,Visa_Master_saldototal_slope_6 =nuevas_col2.Visa_Master_saldototal_slope_6
,Visa_Master_saldototal_slope_4 =nuevas_col2.Visa_Master_saldototal_slope_4
,lag_1_Visa_Master_saldopesos =nuevas_col2.lag_1_Visa_Master_saldopesos
,lag_2_Visa_Master_saldopesos =nuevas_col2.lag_2_Visa_Master_saldopesos
,lag_3_Visa_Master_saldopesos =nuevas_col2.lag_3_Visa_Master_saldopesos
,lag_4_Visa_Master_saldopesos =nuevas_col2.lag_4_Visa_Master_saldopesos
,delta_1_Visa_Master_saldopesos =nuevas_col2.delta_1_Visa_Master_saldopesos
,avg_6_Visa_Master_saldopesos =nuevas_col2.avg_6_Visa_Master_saldopesos
,avg_4_Visa_Master_saldopesos =nuevas_col2.avg_4_Visa_Master_saldopesos
,Visa_Master_saldopesos_slope_6 =nuevas_col2.Visa_Master_saldopesos_slope_6
,Visa_Master_saldopesos_slope_4 =nuevas_col2.Visa_Master_saldopesos_slope_4

from nuevas_col2
where nuevas_col.numero_de_cliente = nuevas_col2.numero_de_cliente and nuevas_col.foto_mes = nuevas_col2.foto_mes;

,Success


In [ ]:
columnas_new = segunda_todas_col

sql_query = ""
for i in columnas_new:
    sql_query += f"\n alter table nuevas_col add column {i} double precision;"
print(sql_query)

In [125]:
columnas_new = segunda_todas_col

sql_query = ""
for i in columnas_new:
    sql_query += f"\n alter table nuevas_col add column {i} double precision;"
print(sql_query)

,numero_de_cliente,foto_mes,cliente_antiguedad,Visa_msaldototal,Master_msaldototal,Visa_Master_saldototal,Master_Fvencimiento,Visa_Fvencimiento,tc_fvencimiento_mayor,tc_fvencimiento_menor,...,Visa_Master_saldototal_slope_4,lag_1_Visa_Master_saldopesos,lag_2_Visa_Master_saldopesos,lag_3_Visa_Master_saldopesos,lag_4_Visa_Master_saldopesos,delta_1_Visa_Master_saldopesos,avg_6_Visa_Master_saldopesos,avg_4_Visa_Master_saldopesos,Visa_Master_saldopesos_slope_6,Visa_Master_saldopesos_slope_4
0,29188883,201901,296,956.35,0.0,956.35,-1276,-2007,-1276,-2007,...,NaN,NaN,NaN,NaN,NaN,NaN,1121.790000,1121.790000,NaN,NaN
1,29188883,201902,297,940.53,0.0,940.53,-1248,-1979,-1248,-1979,...,-15.820,1121.79,NaN,NaN,NaN,-18.55,1112.515000,1112.515000,-18.550000,-18.550
2,29188883,201903,298,1148.28,0.0,1148.28,-1217,-1948,-1217,-1948,...,95.965,1103.24,225.14,NaN,NaN,243.69,1190.653333,1190.653333,112.570000,112.570
3,29188883,201904,299,1128.41,0.0,1128.41,-1187,-1918,-1187,-1918,...,72.393,1346.93,220.38,201.83,NaN,-23.31,1223.895000,1223.895000,84.918000,84.918
4,29188883,201905,300,1183.35,0.0,1183.35,-1156,-1887,-1156,-1887,...,64.188,1323.62,41.13,284.82,266.27,64.44,1256.728000,1256.728000,75.292000,75.292
5,29188883,201906,301,1381.77,0.0,1381.77,-1126,-1857,-1126,-1857,...,91.755,1388.06,297.19,273.88,517.57,232.75,1317.408333,1356.532000,95.035714,107.627
6,29188883,201907,302,1183.35,0.0,1183.35,-1095,-1826,-1095,-1826,...,32.350,1620.81,0.00,64.44,41.13,-232.75,1327.501429,1413.496000,66.967143,37.945
7,29188883,201908,303,1319.70,0.0,1319.70,-1064,-1795,-1064,-1795,...,38.258,1388.06,-72.81,159.94,224.38,159.94,1388.388571,1453.710000,61.204643,44.876
8,29188883,201909,304,1460.11,0.0,1460.11,-1034,-1765,-1034,-1765,...,49.145,1548.00,324.64,91.89,324.64,164.70,1475.454286,1531.526000,55.216786,57.647
9,29188883,201910,305,1348.19,0.0,1348.19,-1003,-1734,-1003,-1734,...,20.960,1712.70,33.42,193.36,-39.39,-131.28,1508.952857,1570.198000,48.209643,24.586


In [ ]:
columnas_new = ["nueva_columna","nueva_col2"]


sql_query = ""
for i in columnas_new:
    sql_query += f"\n alter table competencia_02 add column {i} double precision"
print(sql_query)

In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_ctrx_quarter
from competencia_02
limit 10


In [40]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over part_mes as mes_1
        , lead(mes_0, 2) over part_mes as mes_2
        , CASE 
            WHEN mes_0 = 1 AND mes_1 = 0 AND mes_2=0 THEN 'baja+1'            
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2=0 THEN 'baja+2'
            WHEN mes_0 = 1 AND mes_1 = 1 AND mes_2=1 THEN 'continua'
            ELSE 'raros'
          END as clase_ternaria 
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
    WINDOW part_mes AS (partition by t.numero_de_cliente order by foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1

,Success


In [41]:
%%sql
SELECT clase_ternaria, COUNT(*) AS conteo
FROM targets
WHERE foto_mes NOT IN ('202107','202106')
GROUP BY clase_ternaria


,clase_ternaria,conteo
0,continua,4195878
1,baja+2,18987
2,baja+1,18783
3,raros,60


In [102]:
#agregar las nuevas columnas en la tabla original

columnas_new = segunda_todas_col

sql_query = ""
for i in columnas_new:
    sql_query += f"\n alter table competencia_02 add column {i} double precision"
print(sql_query)



 alter table competencia_02 add column lag_1_Visa_Master_finlim double precision
 alter table competencia_02 add column lag_2_Visa_Master_finlim double precision
 alter table competencia_02 add column lag_3_Visa_Master_finlim double precision
 alter table competencia_02 add column lag_4_Visa_Master_finlim double precision
 alter table competencia_02 add column delta_1_Visa_Master_finlim double precision
 alter table competencia_02 add column avg_6_Visa_Master_finlim double precision
 alter table competencia_02 add column avg_4_Visa_Master_finlim double precision
 alter table competencia_02 add column Visa_Master_finlim_slope_6 double precision
 alter table competencia_02 add column Visa_Master_finlim_slope_4 double precision
 alter table competencia_02 add column lag_1_Visa_Master_saldototal double precision
 alter table competencia_02 add column lag_2_Visa_Master_saldototal double precision
 alter table competencia_02 add column lag_3_Visa_Master_saldototal double precision
 alter ta

In [46]:
%%sql 
alter table competencia_02 add column clase_ternaria VARCHAR(10)

,Success


In [48]:
%%sql
copy competencia_02 to './csv/competencia_02_b6.csv.gz' (FORMAT CSV, HEADER)


,Success


Para cargar el archivo `.csv` a una tabla:

In [ ]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('/home/aleb/dmeyf23/datasets/competencia_01_crudo.csv')

Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [ ]:
%sql select * from competencia_01 limit 5

In [ ]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [ ]:
%%sql
-- escriba la query 

## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [ ]:
%%sql
-- introduzca sus queries

Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [ ]:
%%sql
COPY competencia_01 TO '/home/aleb/dmeyf23/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo

In [1]:
import pandas as pd


In [2]:
df_crudo = pd.read_csv(".\csv\competencia_01_robado.csv", sep= ",")

C:\Users\lrktl\AppData\Local\Temp\ipykernel_9376\1562798170.py:1: DtypeWarning: Columns (154) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crudo = pd.read_csv(".\csv\competencia_01_robado.csv", sep= ",")


In [25]:
df_crudo.isna().sum()

numero_de_cliente               0
foto_mes                        0
active_quarter                  0
cliente_vip                     0
internet                        0
                            ...  
Visa_mconsumototal          64909
Visa_cconsumos              64909
Visa_cadelantosefectivo     64909
Visa_mpagominimo            23808
clase_ternaria             327739
Length: 155, dtype: int64

In [46]:
valores = df_crudo[df_crudo["foto_mes"]==202103].isna().sum()

In [47]:
valores

numero_de_cliente              0
foto_mes                       0
active_quarter                 0
cliente_vip                    0
internet                       0
                           ...  
Visa_mconsumototal         21839
Visa_cconsumos             21839
Visa_cadelantosefectivo    21839
Visa_mpagominimo            7831
clase_ternaria                 0
Length: 155, dtype: int64

In [34]:
for count, i in enumerate(valores):
    print(valores.index[count])

numero_de_cliente
foto_mes
active_quarter
cliente_vip
internet
cliente_edad
cliente_antiguedad
mrentabilidad
mrentabilidad_annual
mcomisiones
mactivos_margen
mpasivos_margen
cproductos
tcuentas
ccuenta_corriente
mcuenta_corriente_adicional
mcuenta_corriente
ccaja_ahorro
mcaja_ahorro
mcaja_ahorro_adicional
mcaja_ahorro_dolares
cdescubierto_preacordado
mcuentas_saldo
ctarjeta_debito
ctarjeta_debito_transacciones
mautoservicio
ctarjeta_visa
ctarjeta_visa_transacciones
mtarjeta_visa_consumo
ctarjeta_master
ctarjeta_master_transacciones
mtarjeta_master_consumo
cprestamos_personales
mprestamos_personales
cprestamos_prendarios
mprestamos_prendarios
cprestamos_hipotecarios
mprestamos_hipotecarios
cplazo_fijo
mplazo_fijo_dolares
mplazo_fijo_pesos
cinversion1
minversion1_pesos
minversion1_dolares
cinversion2
minversion2
cseguro_vida
cseguro_auto
cseguro_vivienda
cseguro_accidentes_personales
ccaja_seguridad
cpayroll_trx
mpayroll
mpayroll2
cpayroll2_trx
ccuenta_debitos_automaticos
mcuenta_debitos

In [48]:
ele_null = []
valores = df_crudo.isna().sum()
for count, i in enumerate(valores):    
    if i != 0:
        ele_null.append(df_crudo.isna().sum().index[count])
        


In [3]:
df_crudo[df_crudo['foto_mes']==202103].info(show_counts=True,verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 163324 entries, 245303 to 491062
Data columns (total 155 columns):
 #    Column                                Non-Null Count   Dtype  
---   ------                                --------------   -----  
 0    numero_de_cliente                     163324 non-null  int64  
 1    foto_mes                              163324 non-null  int64  
 2    active_quarter                        163324 non-null  int64  
 3    cliente_vip                           163324 non-null  int64  
 4    internet                              163324 non-null  int64  
 5    cliente_edad                          163324 non-null  int64  
 6    cliente_antiguedad                    163324 non-null  int64  
 7    mrentabilidad                         163324 non-null  float64
 8    mrentabilidad_annual                  163324 non-null  float64
 9    mcomisiones                           163324 non-null  float64
 10   mactivos_margen                       163324 non-null 

In [4]:
df_03 = df_crudo


In [5]:
df_03

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,60462416,202105,1,0,0,57,225,1809.35,18956.93,1782.91,...,26.0,7671.49,-6942.21,0.0,6243.0,4472.26,9.0,0.0,434.01,NaN
1,60462894,202105,1,0,0,39,80,10158.26,126104.58,2594.76,...,12.0,0.00,-22896.96,0.0,2370.0,14886.39,8.0,0.0,25020.09,NaN
2,60463082,202105,1,0,0,40,225,1156.34,20552.73,1597.34,...,5.0,42697.97,-75961.66,0.0,5979.0,24418.52,9.0,0.0,6029.22,NaN
3,60463355,202105,1,0,0,37,78,-2580.29,18234.46,-1530.83,...,5.0,0.00,-23460.00,0.0,2351.0,6146.77,6.0,0.0,12023.25,NaN
4,60464073,202105,1,0,0,41,87,2097.39,18331.40,495.61,...,5.0,0.00,-65517.02,0.0,2630.0,49637.85,9.0,0.0,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491058,116788661,202103,0,0,1,38,58,1702.94,5056.40,1592.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAJA+1
491059,116793272,202103,1,0,0,47,101,3425.38,12409.73,2265.45,...,1.0,0.00,-5191.39,0.0,3046.0,70.49,1.0,0.0,5090.82,BAJA+1
491060,116802862,202103,1,0,0,56,90,2733.40,14425.01,2124.66,...,1.0,0.00,-34427.12,0.0,2716.0,17046.76,6.0,0.0,3002.88,BAJA+1
491061,117098346,202103,1,0,0,56,28,1942.34,23984.15,1776.32,...,1.0,0.00,-5757.31,0.0,807.0,0.00,0.0,0.0,293.25,BAJA+1


In [12]:
df_03['tc_saldo_total']=df_03['Master_msaldototal'].fillna(0)+df_03['Visa_msaldototal'].fillna(0)
df_03[['tc_saldo_total','Master_msaldototal','Visa_msaldototal']]

,tc_saldo_total,Master_msaldototal,Visa_msaldototal
0,6540.06,0.00,6540.06
1,309124.48,112275.48,196849.00
2,36400.66,0.00,36400.66
3,81177.56,12806.79,68370.77
4,44938.79,0.00,44938.79
...,...,...,...
491058,0.00,NaN,NaN
491059,4399.24,0.00,4399.24
491060,31607.63,0.00,31607.63
491061,2277.00,0.00,2277.00


In [15]:
df_03.drop(columns=['Master_msaldototal','Visa_msaldototal'],inplace=True)

In [ ]:
'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional','ccuenta_debitos_automaticos','mcuenta_debitos_automaticos','cpagomiscuentas', 'mpagomiscuentas'

In [21]:
df_03[[ 'Master_mlimitecompra', 'Visa_mlimitecompra' ]]

df_03['suma_limcompr_visa_master']=df_03['Master_mlimitecompra'].fillna(0)+df_03['Visa_mlimitecompra'].fillna(0)
df_03[['suma_limcompr_visa_master','Master_mlimitecompra','Visa_mlimitecompra']]

,suma_limcompr_visa_master,Master_mlimitecompra,Visa_mlimitecompra
0,168912.00,70380.00,98532.00
1,563040.00,281520.00,281520.00
2,1184683.25,590014.03,594669.22
3,900864.00,450432.00,450432.00
4,2339202.32,1169601.16,1169601.16
...,...,...,...
491058,0.00,NaN,NaN
491059,563040.00,281520.00,281520.00
491060,211140.00,70380.00,140760.00
491061,197064.00,98532.00,98532.00


In [22]:
df_03.drop(columns=['Master_mlimitecompra','Visa_mlimitecompra'],inplace=True)


In [24]:
'mcuenta_corriente' 'mcaja_ahorro'

df_03['saldo_prem']=df_03['mcuenta_corriente'].fillna(0)+df_03['mcaja_ahorro'].fillna(0)
df_03[['saldo_prem','mcuenta_corriente','mcaja_ahorro']]

,saldo_prem,mcuenta_corriente,mcaja_ahorro
0,2512.65,0.00,2512.65
1,2758.36,-719.81,3478.17
2,18026.86,-40.75,18067.61
3,9686.32,-6434.50,16120.82
4,92273.24,-7585.70,99858.94
...,...,...,...
491058,-3556.64,-3556.64,0.00
491059,-820.93,-850.98,30.05
491060,5287.12,-36.01,5323.13
491061,9919.73,1049.50,8870.23


In [25]:
df_03.drop(columns=['mcuenta_corriente','mcaja_ahorro'],inplace=True)

In [28]:
df_03.to_csv(path_or_buf=".\csv\competencia_01_fe.csv")